# 1. ChatMessageHistory的使用


## 场景1: 记忆存储

In [2]:
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

history =  ChatMessageHistory()

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

print(history.messages)


[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]


## 场景2: 对接LLM

In [3]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain.memory import ChatMessageHistory

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)


history =  ChatMessageHistory()

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

history.add_user_message("帮我计算1 + 2 + 3 = ?")

response = chat_model.invoke(history.messages)

print(response.content)


按照数学运算顺序，从左到右依次计算：
先计算1 + 2 = 3，再计算3 + 3 = 6。

所以1 + 2 + 3的结果是6。 


# 2. ConversationBufferMemory

- 适用场景：对话轮次较少、依赖完整上下文的场景（如简单的聊天机器）
- 特点：
    - 完整存储对话历史
    - 简单 、 无裁剪 、 无压缩
    - 与 Chains/Models 无缝集成
    - 支持两种返回格式（通过 return_messages 参数控制输出格式）
        - return_messages=True 返回消息对象列表（ List[BaseMessage]
        - return_messages=False （默认） 返回拼接的 纯文本字符串



## 举例1: 以字符串的方式返回存储的信息

In [5]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(inputs={"input":"你好，我叫小明"},outputs = {"output":"很高兴认识你"})

memory.save_context(inputs={"input":"请帮我回到一下1 + 2 * 3 = ?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))
 

{'history': 'Human: 你好，我叫小明\nAI: 很高兴认识你\nHuman: 请帮我回到一下1 + 2 * 3 = ?\nAI: 7'}


## 举例2: 以消息列表的方式返回存储的消息

In [7]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(inputs={"input":"你好，我叫小明"},outputs = {"output":"很高兴认识你"})

memory.save_context(inputs={"input":"请帮我回到一下1 + 2 * 3 = ?"},outputs={"output":"7"})

print(memory.load_memory_variables({}))

print('\n')

print(memory.chat_memory.messages)
 

{'history': [HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我回到一下1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}


[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我回到一下1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]


## 举例3: 结合大模型、提示词模版的使用(promptTemplate)

In [15]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain.memory import ChatMessageHistory
from langchain_core. prompts. prompt import PromptTemplate
from langchain. memory. buffer import ConversationBufferMemory
from langchain. chains. llm import LLMChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template = """
    你可以与人类对话,
    当前对话历史:{history},
    人类问题:{question},
    回复:
    """
)

memory =  ConversationBufferMemory()

chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response =  chain.invoke({"question":"我叫小明"})

print(response)





{'question': '我叫小明', 'history': '', 'text': '你好，小明！很高兴认识你，有什么想聊的都可以跟我说哦。 '}


In [16]:
response =  chain.invoke({"question":"我叫什么名字呢？"})

print(response)


{'question': '我叫什么名字呢？', 'history': 'Human: 我叫小明\nAI: 你好，小明！很高兴认识你，有什么想聊的都可以跟我说哦。 ', 'text': '你叫小明，我记住你的名字啦。要是还有别的话题，也能随时和我分享。 '}


## 举例4: 基于举例3，显示设置memory的key的值

In [17]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. memory. buffer import ConversationBufferMemory
from langchain. chains. llm import LLMChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template = """
    你可以与人类对话,
    当前对话历史:{chat_history},
    人类问题:{question},
    回复:
    """
)
memory =  ConversationBufferMemory(memory_key="chat_history")

chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response =  chain.invoke({"question":"我叫小明"})

print(response)

{'question': '我叫小明', 'chat_history': '', 'text': '你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 '}


In [19]:
response =  chain.invoke({"question":"我叫什么名字呢？"})
print(response)

{'question': '我叫什么名字呢？', 'chat_history': 'Human: 我叫小明\nAI: 你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 \nHuman: 我叫什么名字呢？\nAI: 你叫小明，你之前已经告诉过我啦。', 'text': '你叫小明呀，我一直都记得呢。 '}


## 举例5: 结合大模型、提示词模版的使用(ChatPromptTemplate)

In [23]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. memory. buffer import ConversationBufferMemory
from langchain. chains. llm import LLMChain
from langchain_core. prompts. chat import ChatPromptTemplate
from langchain_core. prompts. chat import MessagesPlaceholder

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  ChatPromptTemplate.from_messages([
    ("system","你是一个与人类对话的机器人"),
    MessagesPlaceholder(variable_name="history"),
    ("human","问题:{question}")
    
]
)
memory =  ConversationBufferMemory(return_messages=True)

chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

response =  chain.invoke({"question":"我叫小明"})

print(response)

{'question': '我叫小明', 'history': [HumanMessage(content='我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀，小明！很高兴认识你，要是你有什么想聊的话题、有什么问题，都可以随时跟我说。 ', additional_kwargs={}, response_metadata={})], 'text': '你好呀，小明！很高兴认识你，要是你有什么想聊的话题、有什么问题，都可以随时跟我说。 '}


In [24]:
response =  chain.invoke({"question":"我叫什么名字呢？"})
print(response)

{'question': '我叫什么名字呢？', 'history': [HumanMessage(content='我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='你好呀，小明！很高兴认识你，要是你有什么想聊的话题、有什么问题，都可以随时跟我说。 ', additional_kwargs={}, response_metadata={}), HumanMessage(content='我叫什么名字呢？', additional_kwargs={}, response_metadata={}), AIMessage(content='你叫小明，刚刚你已经告诉过我啦。', additional_kwargs={}, response_metadata={})], 'text': '你叫小明，刚刚你已经告诉过我啦。'}


# 3. ConversationChain的使用

## 举例1: 以PromptTemplate为例

In [31]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. chains. conversation. base import ConversationChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

prompt_template =  PromptTemplate.from_template(
    template = """
    你可以与人类对话,
    当前对话历史:{history},
    人类问题:{input},
    回复:
    """
)
# memory =  ConversationBufferMemory(return_messages=True)
# 
# chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

# 创建ConversationChain的实例
chain = ConversationChain(llm=chat_model, prompt=prompt_template)

response =  chain.invoke({"input":"我叫小明"})

print(response)

{'input': '我叫小明', 'history': '', 'response': '你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 '}


In [33]:
response =  chain.invoke({"input":"我叫什么名字呢？"})
print(response)

{'input': '我叫什么名字呢？', 'history': 'Human: 我叫小明\nAI: 你好，小明！很高兴认识你，有什么想聊的都可以跟我说。 ', 'response': '你叫小明，之前你已经告诉过我啦。'}


## 举例2: 使用默认的提示词模版

In [2]:
import os
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core. prompts. prompt import PromptTemplate
from langchain. chains. conversation. base import ConversationChain

dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# prompt_template =  PromptTemplate.from_template(
#     template = """
#     你可以与人类对话,
#     当前对话历史:{history},
#     人类问题:{input},
#     回复:
#     """
# )
# memory =  ConversationBufferMemory(return_messages=True)
# 
# chain =  LLMChain(llm=chat_model, prompt=prompt_template, memory=memory)

# 创建ConversationChain的实例
chain = ConversationChain(llm=chat_model)

response =  chain.invoke({"input":"我叫小明"})

print(response)

{'input': '我叫小明', 'history': '', 'response': '你好呀，小明！很高兴认识你。能和我分享一些你生活里有趣的事儿吗，比如你平时有什么特别喜欢的爱好呀？ '}


In [3]:
response =  chain.invoke({"input":"我叫什么？"})

print(response)

{'input': '我叫什么？', 'history': 'Human: 我叫小明\nAI: 你好呀，小明！很高兴认识你。能和我分享一些你生活里有趣的事儿吗，比如你平时有什么特别喜欢的爱好呀？ ', 'response': '你叫小明呀，我记住啦！你要是还有其他想聊的，不管是生活趣事、兴趣爱好，都能接着和我说呢。 '}


# 4. ConversationBufferWindowMemory的使用

## 举例1:

In [5]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = 2)

memory.save_context({"input":"你好"},{"output":"怎么了"})
memory.save_context({"input":"你是谁"},{"output":"我是AI助手"})
memory.save_context({"input":"你的生日是哪天"},{"output":"我不清楚"})

print(memory.load_memory_variables({}))


{'history': 'Human: 你是谁\nAI: 我是AI助手\nHuman: 你的生日是哪天\nAI: 我不清楚'}


## 举例2: 返回消息构成的上下文记忆

In [6]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k = 2,return_messages=True)

memory.save_context({"input":"你好"},{"output":"怎么了"})
memory.save_context({"input":"你是谁"},{"output":"我是AI助手"})
memory.save_context({"input":"你的生日是哪天"},{"output":"我不清楚"})

print(memory.load_memory_variables({}))


{'history': [HumanMessage(content='你是谁', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手', additional_kwargs={}, response_metadata={}), HumanMessage(content='你的生日是哪天', additional_kwargs={}, response_metadata={}), AIMessage(content='我不清楚', additional_kwargs={}, response_metadata={})]}


## 举例3: 结合llm,chain的使用

In [8]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
import os
import dotenv

# 1. 定义对话模板
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
{history}
Human: {question}
AI:"""

# 2. 创建提示词模板
prompt_template = PromptTemplate.from_template(template)

# 3. 初始化大模型 (注意：当前OpenAI没有"gpt-4o-mini"模型，建议使用"gpt-3.5-turbo"或"gpt-4")
dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# 4. 实例化带窗口的记忆 (k=1表示只保留最近1轮对话)
memory = ConversationBufferWindowMemory(k=1)

# 5. 创建对话链
conversation_with_summary = LLMChain(
    llm=chat_model,
    prompt=prompt_template,
    memory=memory,
    verbose=True  # 开启详细日志
)

# 6. 测试对话流程
# 第一次提问 - 提供名字
respon1 = conversation_with_summary.invoke({"question": "你好，我是孙小空"})
print("第一次回复:", respon1['text'])

# 第二次提问 - 提供师弟信息
respon2 = conversation_with_summary.invoke({"question": "我还有两个师弟，一个是猪小戒，一个是沙小僧"})
print("\n第二次回复:", respon2['text'])

# 第三次提问 - 高考信息
respon3 = conversation_with_summary.invoke({"question": "我今年高考，竟然考上了1本"})
print("\n第三次回复:", respon3['text'])

# 第四次提问 - 测试记忆 (由于k=1，这里只能记住最后一次对话)
respon4 = conversation_with_summary.invoke({"question": "我叫什么？"})
print("\n第四次回复:", respon4['text'])



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：

Human: 你好，我是孙小空
AI:

> Finished chain.
第一次回复: 你好呀孙小空，很高兴认识你！要是你有什么想聊的话题，不管是电影、书籍、科技，还是其他方面的，都能跟我说一说。 


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 你好，我是孙小空
AI: 你好呀孙小空，很高兴认识你！要是你有什么想聊的话题，不管是电影、书籍、科技，还是其他方面的，都能跟我说一说。 
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI:

> Finished chain.

第二次回复: 哇，感觉你们的名字很有特色呢，让我一下就联想到了《西游记》里的孙悟空、猪八戒和沙僧。你们是不是也像他们一样有很多有趣的冒险故事呀？能和我说说你们一起经历过的好玩的事儿不？ 


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI: 哇，感觉你们的名字很有特色呢，让我一下就联想到了《西游记》里的孙悟空、猪八戒和沙僧。你们是不是也像他们一样有很多有趣的冒险故事呀？能和我说说你们一起经历过的好玩的事儿不？ 
Human: 我今年高考，竟然考上了1本
AI:

> Finished chain.

第三次回复: 恭喜恭喜呀！考上一本可是非常厉害的事情，这背后一定有你付出的无数努力呢。能跟我说说你查到成绩那一刻的心情吗？还有你有没有心仪的大学和专业呀？ 


> Entering new LLMChain

## 举例4: 修改举例3中的参数k

In [9]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI
import os
import dotenv

# 1. 定义对话模板
template = """以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
{history}
Human: {question}
AI:"""

# 2. 创建提示词模板
prompt_template = PromptTemplate.from_template(template)

# 3. 初始化大模型 (注意：当前OpenAI没有"gpt-4o-mini"模型，建议使用"gpt-3.5-turbo"或"gpt-4")
dotenv.load_dotenv()

chat_model =  ChatOpenAI(
    model_name = os.getenv("MODEL"),
    base_url = os.getenv("BASE_URL"),
    api_key = os.getenv("ARK_API_KEY"),
)

# 4. 实例化带窗口的记忆 (k=1表示只保留最近1轮对话)
memory = ConversationBufferWindowMemory(k=3)

# 5. 创建对话链
conversation_with_summary = LLMChain(
    llm=chat_model,
    prompt=prompt_template,
    memory=memory,
    verbose=True  # 开启详细日志
)

# 6. 测试对话流程
# 第一次提问 - 提供名字
respon1 = conversation_with_summary.invoke({"question": "你好，我是孙小空"})
print("第一次回复:", respon1['text'])

# 第二次提问 - 提供师弟信息
respon2 = conversation_with_summary.invoke({"question": "我还有两个师弟，一个是猪小戒，一个是沙小僧"})
print("\n第二次回复:", respon2['text'])

# 第三次提问 - 高考信息
respon3 = conversation_with_summary.invoke({"question": "我今年高考，竟然考上了1本"})
print("\n第三次回复:", respon3['text'])

# 第四次提问 - 测试记忆 (由于k=1，这里只能记住最后一次对话)
respon4 = conversation_with_summary.invoke({"question": "我叫什么？"})
print("\n第四次回复:", respon4['text'])



> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：

Human: 你好，我是孙小空
AI:

> Finished chain.
第一次回复: 你好呀孙小空，很高兴认识你！有什么问题都可以随时问我呢，不管是天文地理，还是生活琐事，我都会尽力提供详细的解答。 


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 你好，我是孙小空
AI: 你好呀孙小空，很高兴认识你！有什么问题都可以随时问我呢，不管是天文地理，还是生活琐事，我都会尽力提供详细的解答。 
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI:

> Finished chain.

第二次回复: 哇，这名字听起来很有《西游记》的感觉呢！猪小戒对应猪八戒，沙小僧对应沙僧，而你孙小空自然就是对应孙悟空啦。你们这组合感觉就像一支充满奇幻色彩的小队伍，能跟我说说你们平时会一起做些什么有趣的事儿吗？ 


> Entering new LLMChain chain...
Prompt after formatting:
以下是人类与AI之间的友好对话描述。AI表现得很健谈，并提供了大量来自其上下文的
具体细节。如果AI不知道问题的答案，它会表示不知道。

当前对话：
Human: 你好，我是孙小空
AI: 你好呀孙小空，很高兴认识你！有什么问题都可以随时问我呢，不管是天文地理，还是生活琐事，我都会尽力提供详细的解答。 
Human: 我还有两个师弟，一个是猪小戒，一个是沙小僧
AI: 哇，这名字听起来很有《西游记》的感觉呢！猪小戒对应猪八戒，沙小僧对应沙僧，而你孙小空自然就是对应孙悟空啦。你们这组合感觉就像一支充满奇幻色彩的小队伍，能跟我说说你们平时会一起做些什么有趣的事儿吗？ 
Human: 我今年高考，竟然考上了1本
AI:

> Finished chai